In [2]:
import pandas as pd

In [4]:
station_info = pd.read_csv('station_info_zipcode.csv')

In [25]:
# station_info

In [6]:
mta_2018_df = pd.read_pickle('mta_2018')

In [7]:
mta_2018_df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,00:00:00,REGULAR,6616743,2242108
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,04:00:00,REGULAR,6616762,2242113
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,08:00:00,REGULAR,6616781,2242144
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,12:00:00,REGULAR,6616886,2242229
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,16:00:00,REGULAR,6617105,2242278
...,...,...,...,...,...,...,...,...,...,...,...
196738,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,05:00:00,REGULAR,2919546,387039
196739,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,09:00:00,REGULAR,2919876,387048
196740,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,13:00:00,REGULAR,2920314,387055
196741,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,17:00:00,REGULAR,2920790,387062


In [17]:
# reset mta data index
mta_2018_df.reset_index(drop=True,inplace=True)

# rename column EXITS
mta_2018_df.rename(columns={'EXITS                                                               ':'EXITS'},inplace=True)

In [18]:
mta_2018_df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,00:00:00,REGULAR,6616743,2242108
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,04:00:00,REGULAR,6616762,2242113
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,08:00:00,REGULAR,6616781,2242144
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,12:00:00,REGULAR,6616886,2242229
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,16:00:00,REGULAR,6617105,2242278
...,...,...,...,...,...,...,...,...,...,...,...
788281,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,05:00:00,REGULAR,2919546,387039
788282,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,09:00:00,REGULAR,2919876,387048
788283,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,13:00:00,REGULAR,2920314,387055
788284,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,17:00:00,REGULAR,2920790,387062


In [13]:
# prepare for merging
# Change Stop Name to STATION and convert to all uppercase
station_zipcode_df = station_info[["Stop Name","zip_code"]].copy()
station_zipcode_df["STATION"] = station_zipcode_df['Stop Name'].apply(lambda x: x.upper())
station_zipcode_df.drop(columns=["Stop Name"],inplace=True)

In [14]:
station_zipcode_df

,zip_code,STATION
0,11101,ASTORIA-DITMARS BLVD
1,11102,ASTORIA BLVD
2,11101,30 AV
3,11101,BROADWAY
4,11106,36 AV
...,...,...
491,10309,PRINCE'S BAY
492,10309,PLEASANT PLAINS
493,10307,RICHMOND VALLEY
494,10307,TOTTENVILLE


In [15]:
station_zipcode_df.nunique()

zip_code    140
STATION     379
dtype: int64

In [16]:
# some stations have multiple zipcodes
# keep the first one for now. Further investigation?
station_zipcode_unique_df = station_zipcode_df[~station_zipcode_df.STATION.duplicated(keep='first')]
station_zipcode_unique_df.reset_index(drop=True,inplace=True)
station_zipcode_unique_df

,zip_code,STATION
0,11101,ASTORIA-DITMARS BLVD
1,11102,ASTORIA BLVD
2,11101,30 AV
3,11101,BROADWAY
4,11106,36 AV
...,...,...
374,10309,PRINCE'S BAY
375,10309,PLEASANT PLAINS
376,10307,RICHMOND VALLEY
377,10307,TOTTENVILLE


In [21]:
# merge mta and station_info
mta_2018_df.merge(station_zipcode_unique_df, on="STATION", how='left')

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,zip_code
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,00:00:00,REGULAR,6616743,2242108,11204
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,04:00:00,REGULAR,6616762,2242113,11204
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,08:00:00,REGULAR,6616781,2242144,11204
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,12:00:00,REGULAR,6616886,2242229,11204
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/12/2018,16:00:00,REGULAR,6617105,2242278,11204
...,...,...,...,...,...,...,...,...,...,...,...,...
788281,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,05:00:00,REGULAR,2919546,387039,NaN
788282,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,09:00:00,REGULAR,2919876,387048,NaN
788283,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,13:00:00,REGULAR,2920314,387055,NaN
788284,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,06/08/2018,17:00:00,REGULAR,2920790,387062,NaN


In [30]:
# convert DATE to datetime
mta_2018_df['DATE'] = pd.to_datetime(mta_2018_df.DATE)

# categorize weekday/weekend
mta_2018_df['day_of_week'] = mta_2018_df['DATE'].dt.dayofweek
mta_2018_df_weekday = mta_2018_df[mta_2018_df['day_of_week'] < 5]
mta_2018_df_weekend = mta_2018_df[mta_2018_df['day_of_week'] >= 5]